# From serial to parallel

So far, all our algorithms have been "serial". That is, they've involved performing tasks one after the other, as the name suggests. But in our real life experience, we often find that if we enlist the help of our friends, colleagues, or employees, we can complete a large task by breaking it down into smaller tasks and sharing these sub-tasks out. This is often more efficient than a single person taking on the task themselves. Modern computers, including smartphones, now generallly have multiple processing cores, or may be connected to a network of many computers, meaning we can apply this principle to our program design, leading to significant speed boosts.

The simplest kind of parallelism is called "embarrassingly parallel", though don't let the name put you off. In this case, every worker can get on with their tasks individually, without interacting with the other workers. For example, in a basic ballot count for an election, the vote counters need only grab a stack of ballot papers and sort them into piles, and then hand in their piles at the end. The sorting process is embarrassingly parallel. In other situations, however, the workers have to share resources and information -- this will require trickier design and coordination of the tasks. Furthermore, the very act of sharing itself takes valuable time and resources, as will be familiar to anyone who regularly attends meetings at their jobs, meaning that a badly designed parallel algorithm can be slower than it's serial counterpart ("I may as well have done it all myself!"). Needless to say, there is no magic formula for parallel computing.

Python has a bewildering array of libraries available to facilitate this style of computing. We'll study the standard library module <code>multiprocessing</code> to learn the basic concepts, beginning with squeezing more power out of a multi-core computer.

 ## Processes
 
The <code>multiprocessing</code> module provides a class called <code>Process</code>. The Process can then run independently of the rest of the program, by starting its own version of the Python interpreter "in the background", importing the relevant information, and performing the calculation. For this reason, it is important to use the <code>if \__name\__ == '\__main\__':</code> syntax introduced in PurePy 3. to ensure that the module can safely be imported into subprocesses without side-effects.

The Process class has a method called <code>Process.run()</code>, which should contain the procedure the process is to execute when it is started. There are two ways to modify <code>run()</code>: we can either subclass <code>Process</code>, or provide a function as an argument to the <code>\__init\__()</code> method of the class (note that this cannot be a <code>lambda</code> function, in this case).

So, these are essentially equivalent:

In [6]:
import multiprocessing as mp

class MyProcess(mp.Process):
    def run(self):
        print("My only job is to print this message")

if __name__ == '__main__':
    p = MyProcess()
    p.start()

My only job is to print this message


In [4]:
import multiprocessing as mp

def task():
    print("My only job is to print this message")
    
if __name__ == '__main__':
    p = mp.Process(target=task)
    p.start()

My only job is to print this message


How about with arguments in each case?

In [12]:
import multiprocessing as mp

class MyProcess(mp.Process):
    def __init__(self, *numbers):
        # call the usual process initialization function:
        mp.Process.__init__(self)
        # then store the arguments:
        self.numbers = numbers
        
    def run(self):
        print("My only job is to print this message")
        print("and three numbers {}, {}, {}".format(*self.numbers))

if __name__ == '__main__':
    p = MyProcess(1, 2, 3)
    p.start()

My only job is to print this message
and three numbers 1, 2, 3


In [14]:
import multiprocessing as mp

def task(*numbers):
    print("My only job is to print this message")
    print("and three numbers {}, {}, {}".format(*numbers))
          
if __name__ == '__main__':
    p = mp.Process(target=task, args=(1,2,3))
    p.start()

My only job is to print this message
and three numbers 1, 2, 3


So far so good. We've created a process and started it. Note that <code>run()</code> and <code>start()</code> are two different methods; <code>run()</code> is used to define what the process actually does, whereas <code>start()</code> does all the necessary busywork of starting a new Python interpreter, running the function, and then safely closing it down.

So, what's the big deal. Well, we can demonstrate the power of a process by giving it a slightly longer task:

In [15]:
import multiprocessing as mp
import time

class MyProcess(mp.Process):
    def run(self):
        time.sleep(1) # wait 1 second
        print("And yet the process has only just finished")

if __name__ == '__main__':
    p = MyProcess()
    p.start()
    print("I have been printed after the process started")
    

I have been printed after the process started
And yet the process has only just finished


Read this carefully so you can see the magic. The processs <code>p</code> was running <i>in the background</i>, even as the rest of the program was executing.

We can do this several time:

In [17]:
import multiprocessing as mp
import time

class MyProcess(mp.Process):
    def __init__(self, number):
        mp.Process.__init__(self)
        self.number = number
    
    def run(self):
        time.sleep(1) # wait 1 second
        print("And yet process {} has only just finished".format(self.number))

if __name__ == '__main__':
    p,q = MyProcess(1), MyProcess(2)
    p.start()
    q.start()
    print("I have been printed after the process started")

I have been printed after the process started
And yet process 2 has only just finished
And yet process 1 has only just finished


Perhaps more elegantly:

In [21]:
if __name__ == '__main__':
    procs = [MyProcess(x) for x in (1,2,3)]
    [p.start() for p in procs]
    print("I have been printed after the process started")

I have been printed after the process started
And yet process 1 has only just finished
And yet process 3 has only just finished
And yet process 2 has only just finished


As you can see, there isn't really any way to predict the exact order in which the processes will finish their tasks, if they are essentially all doing the same task.

## Getting connected: pipes and queues

The <code>multiprocessing</code> module provides two classes to facilitate the transmission of data from one process to another. These are called pipes, and queues.

### Pipes

The function <code>Pipe()</code> returns a pair of connection objects, each representing the ends of an imaginary pipe that can take information to and from processes. Each connection has <code>send()</code> and <code>recv()</code> methods, which work basically as expected: send puts an object into the pipe, receive pulls an object out of the pipe.

Get ready, because it's a bit of a mind-bender. We'll do lots of examples with variations to get our heads around the behaviour of pipes, and I highly encourage you to play around with this yourself. Let's start saying we have two processes, <code>Alice</code> and <code>Bob</code>. Alice's job will be to send some messages to Bob, whose job is to receive those messages. To facilitate this communication, we'll create a <code>Pipe()</code>.

In [23]:
import multiprocessing as mp

def sender(connection):
    connection.send("A string object")
    print("Sent something down the pipe")
    connection.close()
    
def receiver(connection):
    print("Received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    # since process expects a TUPLE of arguments, we put each end
    # in a single element tuple like (endA,) here
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Alice.start()
    Bob.start()

Sent something down the pipe
Received: 'A string object'


So far, so good, right? Alice put something into the pipe, and then once Bob had been <code>start()</code>ed, he pulled that item out of the pipe. What happens if we add a delay to this process?

In [25]:
import multiprocessing as mp
import time

def sender(connection):
    connection.send("A string object")
    print("Sent something down the pipe")
    connection.close()
    
def receiver(connection):
    print("Received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Alice.start()
    print("Waiting one second")
    time.sleep(1) # add a delay
    Bob.start()

Sent something down the pipe
Waiting one second
Received: 'A string object'


Still fine. The object lingers in the pipe until a process with access to <code>endB</code> receives it. Okay, let's try swapping the order in which we <code>start()</code> Alice and Bob.

In [26]:
import multiprocessing as mp
import time

def sender(connection):
    connection.send("A string object")
    print("Sent something down the pipe")
    connection.close()
    
def receiver(connection):
    print("Bob about to receive")
    print("Received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Bob.start()
    print("Waiting one second")
    time.sleep(1) # add a delay
    Alice.start()
    

Bob about to receive
Waiting one second
Received: 'A string object'
Sent something down the pipe


Okay, what happened here? When Bob called <code>endB.recv()</code>, there was nothing in the pipe. So Bob just waited patiently until there was, and then responded as soon as Alice had been started and sent the object (in fact in this case, Bob confirmed receiving the object before Alice confirmed sending it!).

Next test. Alice will send two objects. Let's see what happens.

In [27]:
import multiprocessing as mp
import time

def sender(connection):
    connection.send("String object 1")
    connection.send("String object 2")
    print("Sent some things down the pipe")
    connection.close()
    
def receiver(connection):
    print("Bob about to receive")
    print("Received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Alice.start()
    print("Waiting one second")
    time.sleep(1) # add a delay
    Bob.start()

Sent some things down the pipe
Waiting one second
Bob about to receive
Received: 'String object 1'


So, Bob didn't receive a tuple or something from the pipe; he just got the first string Alice sent. What happened to the other object Alice sent? It's still there in the pipe:

In [28]:
print(endB.recv())

String object 2


So Bob will have to call <code>recv()</code> for every object in the pipe sent by Alice. 

In [29]:
import multiprocessing as mp
import time

def sender(connection):
    connection.send("String object 1")
    connection.send("String object 2")
    print("Sent some things down the pipe")
    connection.close()
    
def receiver(connection):
    print("Bob about to receive")
    print("Received: '{}'".format(connection.recv()))
    print("Received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Alice.start()
    print("Waiting one second")
    time.sleep(1) # add a delay
    Bob.start()

Sent some things down the pipe
Waiting one second
Bob about to receive
Received: 'String object 1'
Received: 'String object 2'


Right then. Now let's see what happens if Alice tries to receive from the pipe as well. Alice will send two strings down the pipe, Bob will receive one, then Alice will attempt to <code>recv()</code>. Any bets on what's going to happen?

In [34]:
import multiprocessing as mp
import time

def sender(connection):
    connection.send("String object 1")
    connection.send("String object 2")
    print("Sent some things down the pipe")
    time.sleep(1)
    print(connection.recv())
    connection.close()
    
def receiver(connection):
    print("Bob about to receive")
    print("Received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Alice.start()
    print("Waiting one second")
    Bob.start()

Waiting one second
Sent some things down the pipe
Bob about to receive
Received: 'String object 1'



Did you guess right? Alice couldn't get the item from the pipe because it's now at Bob's end. Let's put Alice out of her misery; she's still waiting:

In [35]:
endB.send("") # an empty string for Alice
print(endB.recv()) # get Alice's string object out

String object 2


So, we should think of each connection of the pipe having an "inbox" and "outbox". Stuff from the outbox of one goes to the inbox of the other. You can't put something in the outbox and then collect it from the inbox of the same connection!

However, we can take advantage of the bi-directionality of the pipe:

In [39]:
import multiprocessing as mp
import time

def sender(connection):
    connection.send("Hi Bob, are you there?")
    print("Alice received: '{}'".format(connection.recv()))
    connection.send("Good, everything seems to be working")
    connection.close()
    
def receiver(connection):
    print("Bob Received: '{}'".format(connection.recv()))
    connection.send("Yes Alice, I'm here")
    print("Bob received: '{}'".format(connection.recv()))
    connection.close()
    
if __name__ == '__main__':
    endA, endB = mp.Pipe() # Pipe() spits out two connection objects
    Alice = mp.Process(target=sender, args=(endA,))
    Bob = mp.Process(target=receiver, args=(endB,))
    Alice.start()
    Bob.start()

Bob Received: 'Hi Bob, are you there?'
Alice received: 'Yes Alice, I'm here'
Bob received: 'Good, everything seems to be working'


Of course, these strings are just for demonstrative purposes. Most objects can be sent via the pipe. 

### Queues

If you can get your head around pipes, queues should be fine. A queue is somewhat similar to a pipe, but instead of two connection objects, any number of processes can add things to the queue, or collect something from the queue. The queue has two methods, <code>get()</code> and <code>put()</code>.

In [45]:
import multiprocessing as mp
import time
from random import randint

def putter(Q):
    for x in range(10):
        number = randint(0, 100)
        print("Adding {} to the queue".format(number))
        Q.put(number)

def getter(Q, ID):
    while True:
        item = Q.get()
        if item is None:
            break
        else:
            print("Getter {} got {} from the queue".format(ID, item))
            time.sleep(1)
            
if __name__ == '__main__':
    the_Queue = mp.Queue()
    distributor = mp.Process(target=putter, args=(the_Queue,))
    getters = [mp.Process(target=getter, args=(the_Queue, x)) for x in [1, 2, 3, 4, 5]]
    distributor.start()
    time.sleep(1)
    [g.start() for g in getters]

Adding 34 to the queue
Adding 56 to the queue
Adding 28 to the queue
Adding 17 to the queue
Adding 47 to the queue
Adding 62 to the queue
Adding 15 to the queue
Adding 64 to the queue
Adding 87 to the queue
Adding 30 to the queue
Getter 1 got 34 from the queue
Getter 2 got 56 from the queue
Getter 3 got 28 from the queue
Getter 4 got 17 from the queue
Getter 5 got 47 from the queue
Getter 1 got 62 from the queue
Getter 4 got 15 from the queue
Getter 2 got 64 from the queue
Getter 3 got 87 from the queue
Getter 5 got 30 from the queue


If we look closely, we can see that even though the various workers did not respond precisely in the same order that they were <code>start()</code>ed, the elements came out of the queue in the exact order that they went in. This is called a first-in, first-out (FIFO) queue.


## Pools of workers

The <code>Pool()</code> class spawns a collection of worker processes with a more convenient toolkit to coordinate them. By default, the number of workers is equal to the number of available cores on your computer, though you can provide an integer argument to create more or fewer if you wish.

The 4 methods of the pool we'll look at are <code>apply()</code>,<code>apply_async()</code>, <code>map()</code>, and <code>map_async()</code>.

### <code>apply()</code>

The apply method takes a function as its first argument, and a collection of <code>args</code> or <code>kwds</code> as its 2nd/3rd arguments. The <code>Pool</code> then delegates the execution of this function to one of the workers.

In [1]:
import multiprocessing as mp
import time

def say_something_then_wait(something, wait):
    print(something)
    time.sleep(wait)
    print("Finished waiting")
    
if __name__ == '__main__':
    p = mp.Pool() # will create 4 workers on my quad core machine
    p.apply(say_something_then_wait, args=("This is the first process", 1))
    p.apply(say_something_then_wait, args=("This is the second process", 1))

This is the first process
Finished waiting
This is the second process
Finished waiting


What can we deduce from this? Look closely at the order that everything occurred. There is only one conclusion: the functions were executed serially, rather than in parallel. So, <code>apply()</code> actually isn't used for parallel computing -- the prcoess blocks the rest of the program until it is complete. Thankfully, this is where <code>apply_async</code> comes in.

### <code>apply_async()</code>

Calling <code>apply_async()</code>, with the same arguments as before, will start a process executing that task without blocking the rest of the program. The function itself returns an <code>AsyncResult</code> object. The result can then be extracted from the <code>AsyncResult</code> object when it is needed:

In [12]:
import multiprocessing as mp
import time

def return_a_sentence_after_delay(something, wait):
    print("Process starting, thinking about the word {}".format(something))
    time.sleep(wait)
    sentence = "This sentence contains the word {}.".format(something)
    return sentence
    
if __name__ == '__main__':
    p = mp.Pool()
    
    #note I have given the workers different delays to demonstrate
    result1 = p.apply_async(return_a_sentence_after_delay, args=("HiPy", 1))
    result2 = p.apply_async(return_a_sentence_after_delay, args=("Liverpool", 3))
    
    print("\nWhat is a result?")
    print(type(result1))
    
    time.sleep(1.1) # introduce a delay
    
    print("\nNow checking if results are ready:")
    print("Result 1 ready:", result1.ready()) # method determines if fnctn has finished
    print("Result 2 ready:", result2.ready())
    print("\nNow getting result 1")
    print(result1.get())
    print("\nNow waiting on result 2")
    print(result2.get())
    print("Everything is done now")

Process starting, thinking about the word Liverpool
Process starting, thinking about the word HiPy

What is a result?
<class 'multiprocessing.pool.ApplyResult'>

Now checking if results are ready:
Result 1 ready: True
Result 2 ready: False

Now getting result 1
This sentence contains the word HiPy.

Now waiting on result 2
This sentence contains the word Liverpool.
Everything is done now


Note that we called <code>get()</code> on <code>result2</code> before the function had even finished. The program waited until <code>result2</code> had finished processing. But what if something went wrong in this process? Will the whole program lock up, waiting for a result that will never come?

Thankfully, we can pass an integral argument to <code>get()</code>, for instance <code>get(5)</code>, and the program will wait 5 seconds for a result. If none is forthcoming, then a <code>TimeoutError</code> exception is raised.

### <code>map()</code>

Python has a built-in function called <code>map()</code>, which applies a given function to each element of an iterable object. For example:

In [10]:
def square(x):
    return x*x
print(list(map(square, range(100))))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


There is virtually no reason to use this function, as list comprehensions (explained in PurePy 5) achieve the same thing in a more readable and often more efficient way. The <code>map()</code> function is a vestige from older versions of Python before the introduction of comprehensions.

Nonetheless, the <code>multiprocessing</code> module provides its own version of <code>map()</code>, which takes a function and an iterable. The task is broken up into chunks and distributed to the various workers, potentially leading to a significant speed boost.

In [12]:
import multiprocessing as mp
def square(x):
    return x*x
if __name__ == '__main__':
    p = mp.Pool()
    print(list(p.map(square, range(100))))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


A third argument can be provided to <code>map()</code>, called <code>chunksize</code>, which tells the Pool roughly how big to make the chunks that are distributed to the workers. You can play around with this. I've found that sometimes it improves performance and sometimes worsens it. The idea is that if you have a large amount of data to get through, using chunks that are too small wastes time, as the processes are spending as much time getting new chunks as they are computing.

The method <code>map_async()</code> works much the same as the <code>apply_async()</code>, returning instead a Result object from which the result can be extracted when it is ready.

## Exercise

I'm afraid it's prime checking again. Here I have a very naïve algorithm for filtering the primes out of a sequence of numbers.

In [ ]:
def isprime(n):
    if n == 2  or n == 3:
        return True
    if n == 1 or n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    w = 2
    while i*i <= n:
        if n % i == 0:
            return False
        i += w
        w = 6 - w
    return True

data_range = 1000000

outputs = [x for x in range(data_range) if isprime(x)]

If you paste this into a module, head over to IPython and run it with

or whatever you called it. The <code>-t</code> option times the running of this algorithm. The result "wall time" is the best indicator -- it's the time as measured by a clock on the wall. Run this a few times to get a good feel for how long it takes.

For me, it takes almost precisely 5 seconds to run this. Yours will differ

You challenge? Without modifying the function <code>isprime()</code> or the range of integers, use the parallel computing concepts explained here to cut this time down as much as possible! I can get it down to about 3.5 seconds, but I'm sure you can do better.

Your times will vary enormously with the power of your computer.